## <span style="color:#956bbf">RFM: Recency, Frequency, Monetary value Summary</span>
---

### <span style="color:#956bbf">Introduction</span>
---

**Source**:
- [Creating an RFM Summary Using Excel](https://www.brucehardie.com/notes/022/)
- [Notes on the CDNOW Master Data Set](https://www.brucehardie.com/notes/026/)

In order to estimate the parameters of **transaction-flow models** such as the Pareto/NBD and BG/NBD, as well as those of the associated models for **spend per transaction** (spend model), we need an RFM (recency, frequency, monetary value) summary of each customer’s purchasing behavior. In particular,
1. The **transaction-flow model** requires three pieces of information about each customer’s purchasing history: their “recency”(when their last transaction occurred), “frequency” (how many transactions they made in a specified time period), and the length of time over which we have observed their purchasing behavior. The notation used to represent this information is $(x, tx, T)$, where $x$ is the number of transactions observed in the time period $(0, T]$ and $t_{x}$ $(0 \le tx \le T)$ is the time of the last transaction.
2. The **spend model** requires two pieces of information about each customer’s purchasing history: the average “monetary value” of each transaction (denoted by $m_x$) and the number of transactions over which this average is computed (i.e., frequency, $x$).

### <span style="color:#956bbf">Imports</span>
---

#### Import Packages

In [1]:
import polars as pl
import numpy as np

#### Import Data

We will make use of the CDNOW dataset. The master dataset contains the entire purchase history up to the end of June 1998 of the cohort of 23,570 individuals who made their first-ever purchase at CDNOW in the first quarter of 1997. 

The file `CDNOW_sample.csv` contains purchasing data for a 1/10th systematic sample of the whole cohort (2357 customers). Each record in this file, 6919 in total, comprises five fields: the customer’s ID in the master dataset, the customer’s ID in the 1/10th sample dataset (ranging from 1 to 2357), the date of the transaction, the number of CDs purchased, and the dollar value of the transaction.

In [2]:
CDNOW_master = (
    pl.scan_csv(source = 'data/CDNOW/CDNOW_master.csv', 
                has_header=False, 
                separator=',', 
                schema={'CustID': pl.Int32,     # customer id
                        'Date': pl.String,      # transaction date
                        'Quant': pl.Int16,      # number of CDs purchased
                        'Spend': pl.Float64})   # dollar value (excl. S&H)
    .with_columns(pl.col('Date').str.to_date("%Y%m%d"))
    .with_columns((pl.col('Date') - pl.date(1996,12,31)).dt.total_days().cast(pl.UInt16).alias('PurchDay'))
    .with_columns((pl.col('Spend')*100).round(0).cast(pl.Int64).alias('Spend Scaled'))
    .group_by('ID', 'Date')
    .agg(pl.col('*').exclude('PurchDay').sum(), pl.col('PurchDay').max()) # Multiple transactions by a customer on a single day are aggregated into one
    .sort('ID', 'Date')
    .with_columns((pl.col("CustID").cum_count().over("CustID") - 1).cast(pl.UInt16).alias("DoR"))  # DoR = Depth of Repeat ('Transaction' time: starts with 0 as trial, 1 as 1st repeat and so on)
)

In [3]:
# MATLAB Sampling (due to numerical float precision handling differences, original sampling results cannot be replicated unless spend is scaled in MATLAB)
CDNOW_sample = (
    pl.scan_csv(source='data/CDNOW/CDNOW_sample.csv',
                has_header=False,
                separator=',',
                schema={'CustID': pl.Int32,
                        'NewID': pl.Int32,
                        'Date': pl.String,
                        'Quant': pl.Int16,
                        'Spend': pl.Float64})
    .with_columns(pl.col('Date').str.to_date("%Y%m%d"))
    .with_columns((pl.col('Date') - pl.date(1996,12,31)).dt.total_days().cast(pl.UInt16).alias('PurchDay'))
    .with_columns((pl.col('Spend')*100).round(0).cast(pl.Int64).alias('Spend Scaled'))
    .group_by('NewID', 'Date')
    .agg(pl.col('*').exclude('PurchDay').sum(), pl.col('PurchDay').max())
    .sort('NewID', 'Date')
    .with_columns((pl.col("NewID").cum_count().over("NewID") - 1).cast(pl.UInt16).alias("DoR"))      
    .drop('CustID')
    .rename({'NewID': 'ID'})
)

1. We assume that the records in the raw transaction data file are grouped by customer, and sorted within customer by date of transaction. If in doubt, sort the raw dataset by customer ID and date of transaction.
2. As they are of no interest to us in this particular case, we delete the first and fourth columns (master dataset customer ID and # CDs purchased, respectively). 
3. We note that some customers had more than one transaction on a given day. For example, customer 26 had two separate transactions on 13 January 1997, while customer 46 had two separate transactions on 28 August 1997. There are about 233 such "additional" transactions. The transaction-flow models are developed by telling a story about **interpurchase times**. As we only know the date (and not the time) of each transaction, we need to aggregate the records associated with same-day transactions—we can’t have an interpurchase time of 0

### <span style="color:#956bbf">“Frequency” and “Monetary Value”</span>
---

We now compute the frequency and monetary value summaries for each customer.

Most of the previous analyses undertaken using this dataset have split the 78 weeks of data in half, creating a 39-week calibration period (1997-01-01 to 1997-09-30) and 39-week validation period (1997-10-01 to 1998-06-30). Furthermore, these analyses have generally ignored each customer’s first-ever purchase at CDNOW, which signals the start of the customer’s “relationship” with the firm; this means calibration-period “frequency” has usually been the number of repeat transactions, and “monetary value” has been the average dollar value per repeat transaction.

In [4]:
calwk = 273 # 39 week calibration period

CDNOW_sample = (
    CDNOW_sample
    .with_columns(
        pl.when(pl.col('DoR') == 0)
        .then(pl.lit('first'))
        .otherwise(pl.when(pl.col('PurchDay') <= calwk)
                   .then(pl.lit('calib'))
                   .otherwise(pl.lit('valid'))
        ).alias('Period')
    )
)

In [5]:
# the number of repeat transactions made in the calibration and validation periods
num_repeat_trans = (
    CDNOW_sample
    .group_by('ID', 'Period')
    .agg(pl.len().alias('x'))
)

In [6]:
# the average spend per repeat transaction for the calibration and validation periods
avg_spend_per_repeat_trans = (
    CDNOW_sample
    .group_by('ID', 'Period')
    .agg(pl.col('Spend Scaled').mean().alias('m_x'))
)

### <span style="color:#956bbf">“Recency”</span>
---

The next step is to compute recency, as well as the length of time over which we have observed each customer’s purchasing behavior.

$T$ is the effective calibration period, the length of time we observe each customer (i.e., the time between the customer’s first-ever purchase at CDNOW and the end of the calibration period)

In [16]:
# Effective calibration period - the length of time we observe each customer
eff_cal_period = (
    CDNOW_sample
    .filter(pl.col('DoR') == 0)
    .with_columns(((pl.lit(calwk) - pl.col('PurchDay')) / 7).alias('T')) # Eff. Calibration Period
    .rename({'PurchDay': 'Trial Day'})   
)

For modeling purposes, “recency” is not the calendar date of the last observed purchase; rather the time origin for $t_x$ is the start of the observation period. Since we track customers’ purchasing from their first-ever purchase at CDNOW, the date of this first purchase is the time origin. Therefore, $t_x$ is the length of time between the first-ever purchase and the last observed purchase (in the calibration period), i.e., Last Purchase − First Purchase.

In [18]:
# time of last calibration period repeat purchase (in weeks) - recency number in days
ttlrp = (
    CDNOW_sample
    .filter(pl.col('Period') != 'valid')
    .group_by('ID')
    .agg(pl.col('PurchDay').max())
    .join(other=eff_cal_period, on='ID', how='left')
    .with_columns(((pl.col('PurchDay') - pl.col('Trial Day'))/7).alias('t_x')) # Time to Last Repeat
    .select('ID', 'T', 't_x')
)

### <span style="color:#956bbf">RFM Summary</span>
---

RFM summary of calibration-period transactions:

This is the required “RFM” summary of calibration-period buying behavior needed to estimate the previously mentioned models of *transaction flow* and *spend per transaction*.

In [9]:
rfm = (
    CDNOW_sample
    .group_by('ID').agg()
    .join(
        other=num_repeat_trans.filter(pl.col('Period') == 'calib').drop('Period'), 
        on='ID', how='left'
    ).join(
        other=avg_spend_per_repeat_trans.filter(pl.col('Period') == 'calib').drop('Period'), 
        on='ID', how='left'
    ).join(
        other=ttlrp, 
        on='ID', how='left'        
    ).fill_null(0)
)

We can create a summary of validation-period purchasing behavior by extracting the corresponding
number of transactions and average spend per transaction numbers from the dataframes constructed above.

#### The number of repeat transactions made by each customer in each period

In [10]:
calwk = 273 # 39 week calibration period

# The number of repeat transactions made by each customer in each period
px = (
    CDNOW_sample
    .group_by('ID', maintain_order=True)
    .agg(
        pl.col('PurchDay')
        .filter((pl.col('PurchDay') <= calwk) & (pl.col('DoR') > 0))
        .count()
        .alias('P1X'), # Period 1: Calibration Period

        pl.col('PurchDay')
        .filter((pl.col('PurchDay') > calwk) & (pl.col('DoR') > 0))
        .count()
        .alias('P2X')  # Period 2: Longitudinal Holdout Period      
    )
)

px.collect()

ID,P1X,P2X
i32,u32,u32
1,2,1
2,1,0
3,0,0
4,0,0
5,0,0
…,…,…
2353,0,2
2354,5,0
2355,0,1


#### The number of CDs purchased and total spend across these repeat transactions

In [11]:
# The number of CDs purchased and total spend across these repeat transactions
pSpendQuant = (
    CDNOW_sample
    .join(px, on='ID', how='left')
    .group_by('ID', maintain_order=True)
    .agg(
        
        pl.col('Spend Scaled')
        .filter((pl.col('DoR') > 0) & (pl.col('DoR') <= pl.col('P1X')) & (pl.col('P1X') != 0))
        .sum()
        .alias('P1X Spend'),
        
        pl.col('Quant')
        .filter((pl.col('DoR') > 0) & (pl.col('DoR') <= pl.col('P1X')) & (pl.col('P1X') != 0))
        .sum()
        .alias('P1X Quant'),        
        
        pl.col('Spend Scaled')
        .filter((pl.col('DoR') > 0) & (pl.col('DoR') > pl.col('P1X')))
        .sum()
        .alias('P2X Spend'),
        
        pl.col('Quant')
        .filter((pl.col('DoR') > 0) & (pl.col('DoR') > pl.col('P1X')))
        .sum()
        .alias('P2X Quant')                
    )
)

pSpendQuant.collect()

ID,P1X Spend,P1X Quant,P2X Spend,P2X Quant
i32,i64,i64,i64,i64
1,4469,3,2648,2
2,1177,1,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
…,…,…,…,…
2353,0,0,3955,3
2354,22464,11,0,0
2355,0,0,2460,2


#### The average spend per repeat transaction

In [12]:
# The average spend per repeat transaction
mx = (
    pSpendQuant
    .join(px, on='ID', how='left')
    .with_columns(
        (pl.col('P1X Spend') / pl.col('P1X')).alias('Avg Spend per Repeat')
    ).fill_nan(0)
)

mx.collect()

ID,P1X Spend,P1X Quant,P2X Spend,P2X Quant,P1X,P2X,Avg Spend per Repeat
i32,i64,i64,i64,i64,u32,u32,f64
1,4469,3,2648,2,2,1,2234.5
2,1177,1,0,0,1,0,1177.0
3,0,0,0,0,0,0,0.0
4,0,0,0,0,0,0,0.0
5,0,0,0,0,0,0,0.0
…,…,…,…,…,…,…,…
2353,0,0,3955,3,0,2,0.0
2354,22464,11,0,0,5,0,4492.8
2355,0,0,2460,2,0,1,0.0


#### Recency & effective calibration period

When fitting models such as the Pareto/NBD and BG/NBD to these data, we also want to know the “recency” information for each customer, as well as their effective calibration period:

In [13]:
# time of last calibration period repeat purchase (in weeks)
ttlrp = (
    CDNOW_sample
    .join(px, on='ID', how='left')
    .with_columns(
        pl.col('PurchDay').filter(pl.col('DoR') == 0)
        .first()
        .over('ID')
        .alias('Trial Day')
    )
    .group_by('ID', maintain_order=True)
    .agg(
        pl.col('PurchDay', 'Trial Day')
        .filter(pl.col('DoR') <= pl.col('P1X'))
        .max()
        # .alias('LastPurch')
    )
    .with_columns(
        # effective calibration period (in weeks)
        ((pl.col('PurchDay') - pl.col('Trial Day')) / 7).alias('Time to Last Repeat'),
        ((calwk - pl.col('Trial Day'))/7).alias('Eff. Calibration Period')
    )
    .drop('PurchDay', 'Trial Day')
)

ttlrp.collect()

ID,Time to Last Repeat,Eff. Calibration Period
i32,f64,f64
1,30.428571,38.857143
2,1.714286,38.857143
3,0.0,38.857143
4,0.0,38.857143
5,0.0,38.857143
…,…,…
2353,0.0,27.0
2354,24.285714,27.0
2355,0.0,27.0
